In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

# https://mvnrepository.com/artifact/io.acryl/datahub-spark-lineage

token = 'xxxxxxxxxx'

spark = (SparkSession.builder
         .appName('sample_spark')
         .config('spark.extraListeners', 'datahub.spark.DatahubSparkListener')
         .config('spark.jars.packages', 'io.acryl:datahub-spark-lineage:0.13.1-3')
         .config("spark.datahub.rest.server", "http://datahub-datahub-gms.datahub:8080")
         .config("spark.datahub.rest.token", token)
         .getOrCreate()
        )



In [2]:
df_produtos = spark.read.format('parquet').load('tables/produtos')

df_produtos.show()

+----------+------------+
|produto_id|produto_nome|
+----------+------------+
|         1|       Arroz|
|         2|      Feijão|
|         4|        Óleo|
+----------+------------+



In [3]:
df_vendas = spark.read.format('parquet').load('tables/vendas')

df_vendas.show()

+----------+----------+-----+
|      data|produto_id|valor|
+----------+----------+-----+
|2022-01-01|         1|100.0|
|2022-01-02|         2| 50.0|
|2022-01-03|         3| 25.0|
+----------+----------+-----+



In [4]:
df_final = df_vendas.join(df_produtos, "produto_id", "left")

df_final.show()

+----------+----------+-----+------------+
|produto_id|      data|valor|produto_nome|
+----------+----------+-----+------------+
|         1|2022-01-01|100.0|       Arroz|
|         2|2022-01-02| 50.0|      Feijão|
|         3|2022-01-03| 25.0|        null|
+----------+----------+-----+------------+



In [5]:
df_final.write.mode('overwrite').format("parquet").save("tables/final")